In [6]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display
from bs4 import BeautifulSoup
from openai import OpenAI

In [13]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print('couldn\'t catch API key')

In [29]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self,url):
        self.url =  url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else ''
        self.content = soup.body
        for irrelevent in soup.body(["script", "style", "img", "input"]):
            irrelevent.decompose()
        self.text = soup.body

In [30]:
a = Website('https://www.nytimes.com/international/')
a.t